In [1]:
import aesara 
import aesara.tensor as at
import arviz as az
import matplotlib.pyplot as plt
import numpy as np 
import pymc as pm 
import xarray as xr 
import pandas as pd 
import pymc.sampling_jax
import numpyro 

print(f"Running on PyMC v{pm.__version__}")

Running on PyMC v4.1.3


/Users/conor/miniconda3/envs/pymc_non_dev/lib/python3.10/site-packages/pymc/sampling_jax.py:36: UserWarning: This module is experimental.
  warnings.warn("This module is experimental.")


#### Data: loading in the Scottish lip cancer data

In [2]:
df_scot_cancer = pd.read_csv(pm.get_data("scotland_lips_cancer.csv"))

# observed cancer counts
y = df_scot_cancer["CANCER"].values

# number of observations
N = len(y)

# expected cancer counts E for each county: this is calculated using age-standardized rates of the local population
E = df_scot_cancer["CEXP"].values
logE = np.log(E)

# proportion of the population engaged in agriculture, forestry, or fishing
x = df_scot_cancer["AFF"].values / 10.0

# spatial adjacency information: column `ADJ` contains list entries which are preprocessed to obtain adj as a list of lists
adj = (
    df_scot_cancer["ADJ"].apply(lambda x: [int(val) for val in x.strip("][").split(",")]).to_list()
)

# change to Python indexing (i.e. -1)
for i in range(len(adj)):
    for j in range(len(adj[i])):
        adj[i][j] = adj[i][j] - 1

# storing the adjacency matrix as a two-dimensional np.array
adj_matrix = np.zeros((N, N), dtype="int32")

for area in range(N):
    adj_matrix[area, adj[area]] = 1

In [3]:
# getting the necessary information to use for a spatial model specification 
N_edges = (adj_matrix == 1).sum()
node1 = np.where(adj_matrix == 1)[0] 
node2 = np.where(adj_matrix == 1)[1] 

In [4]:
coords = {"num_areas": N, 
          "num_neighbours": N_edges}

####  Leroux model

We now use a Leroux model such that the random effect structure for each area $i$ is $\psi_i=\phi_i$, where the vector $\boldsymbol{\phi}$ is distributed according to 
\begin{align}
\boldsymbol{\phi}\sim\text{Normal}(0, [\tau^2\rho(\mathbf{D}-\mathbf{W}) + (1-\rho)\mathbf{I})\big ]^{-1}). 
\end{align} 
When $\rho=1$, the Leroux prior is equal to an ICAR prior, and when $\rho=0$, it is equal to an independent random effect. 

In [5]:
# creating the pairwise specification for the Leroux model
def pairwise_diff_leroux(rho, phi, node1, node2):
    return -0.5 * rho * ((phi[node1]-phi[node2]) ** 2).sum()
# creating the square sum potential specification 
def square_sum(rho, phi):
    return -0.5 * (1-rho) * (phi ** 2).sum()

In [6]:
with pm.Model(coords={"num_areas": np.arange(N)}) as leroux_model:
    # precision priors, transform to standard deviation 
    tau_phi = pm.Gamma("tau_phi", alpha=1, beta=1)
    # transform to standard deviation 
    sigma_phi = pm.Deterministic("sigma_phi", 1/at.sqrt(tau_phi))
    # spatial smoothing effect prior 
    rho = pm.Beta("rho",  alpha=1, beta=1)

    # spatial random effect
    phi = pm.Flat("phi", dims="num_areas")
    pm.Potential("spatial_diff", pairwise_diff_leroux(rho, phi, node1, node2))
    pm.Potential("square_sum", square_sum(rho, phi))

    # constraint on the spatial random effect 
    zero_constraint = pm.Normal.dist(mu=0.0, sigma=np.sqrt(0.001))
    pm.Potential("zero_sum_phi", pm.logp(zero_constraint, pm.math.sum(phi))) 

    # regression coefficient priors 
    beta0 = pm.Normal("beta0", mu=0, sigma=5)
    beta1 = pm.Normal("beta1", mu=0, sigma=5)
    
    # linear predictor 
    eta = pm.Deterministic("eta", logE + beta0 + beta1*x + sigma_phi*phi, dims="num_areas") 

    # likelihood
    obs = pm.Poisson("obs", at.exp(eta), observed=y, dims="num_areas")

In [7]:
with leroux_model:
    idata = pm.sample()

Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau_phi, rho, phi, beta0, beta1]
INFO:pymc:NUTS: [tau_phi, rho, phi, beta0, beta1]


/Users/conor/miniconda3/envs/pymc_non_dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/conor/miniconda3/envs/pymc_non_dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/conor/miniconda3/envs/pymc_non_dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/conor/miniconda3/envs/pymc_non_dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 260 seconds.
INFO:pymc:Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 260 seconds.
